In [0]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
spark = SparkSession.builder.appName('recommender').getOrCreate()

In [0]:
data = spark.read.csv("/FileStore/tables/movielens_ratings.csv", inferSchema=True, header=True)

In [0]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [0]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [0]:
 train_data, test_data = data.randomSplit([0.8, 0.2])

In [0]:
als = ALS(maxIter=5,regParam=0.01,userCol='userId',itemCol='movieId',ratingCol='rating')

In [0]:
model = als.fit(train_data)

In [0]:
predictions = model.transform(test_data)

In [0]:
predictions.show()

+-------+------+------+------------+
|movieId|rating|userId|  prediction|
+-------+------+------+------------+
|      4|   2.0|     1|   2.4167862|
|      0|   1.0|     3|   2.0548916|
|      2|   1.0|     3|  0.14842248|
|      0|   1.0|     5|-0.008182406|
|      1|   1.0|     6|   1.9924512|
|      3|   1.0|     9|   1.4632483|
|      5|   1.0|     9|   1.0458877|
|      5|   1.0|    13|    1.344014|
|      3|   3.0|    14|   1.4318928|
|      2|   1.0|    15|    2.402055|
|      2|   1.0|    17|     0.55883|
|      2|   1.0|    19| 0.027450621|
|      0|   1.0|    20|  0.43030575|
|      1|   1.0|    20|    1.095768|
|      4|   2.0|    20|  0.42722252|
|      0|   1.0|    22|  -1.0684953|
|      3|   2.0|    22|   0.4364947|
|      0|   1.0|    23|   3.7838283|
|      4|   1.0|    23|   1.9517736|
|      3|   1.0|    26|   -0.447684|
+-------+------+------+------------+
only showing top 20 rows



In [0]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

In [0]:
rmse = evaluator.evaluate(predictions)
print(rmse)

1.931706904266926


This is a tiny dataset. The model does not perform well.

# Let's check out recommendations for each user

In [0]:
single_user = test_data.filter(test_data['userId'] == 2).select(['movieId', 'userId'])

In [0]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      8|     2|
|      9|     2|
|     15|     2|
|     18|     2|
|     35|     2|
|     55|     2|
|     80|     2|
|     85|     2|
+-------+------+



In [0]:
recommendations = model.transform(single_user)
recommendations.orderBy('prediction', ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|      9|     2|  3.4580767|
|     85|     2|  3.3875136|
|     55|     2| 0.93479836|
|     15|     2| 0.85604644|
|     35|     2|-0.17401612|
|      8|     2|-0.77921724|
|     80|     2| -1.0366478|
|     18|     2| -2.9458377|
+-------+------+-----------+

